## Table of Contents
1. [Setup and Imports](#setup)
2. [Discover Delay Propagation Artifacts](#discover)
3. [Load and Inspect Delay Data](#load)
4. [Cascade Size Distribution](#cascade-dist)
5. [Superspreader Ranking](#superspreaders)
6. [Centrality Overlap Analysis](#overlap)
7. [Interpretation](#interpretation)
8. [Write Report Outputs](#write-outputs)
9. [Reproducibility Notes](#reproducibility)

In [ ]:
# ============================================================================
# SETUP AND IMPORTS
# ============================================================================

import json
from pathlib import Path
from datetime import datetime
import warnings

import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Project paths
REPO_ROOT = Path.cwd().parent.parent
RESULTS_DIR = REPO_ROOT / "results"
ANALYSIS_DIR = RESULTS_DIR / "analysis"
TABLES_REPORT_DIR = RESULTS_DIR / "tables" / "report"
FIGURES_REPORT_DIR = RESULTS_DIR / "figures" / "report"
WARNINGS_LOG = TABLES_REPORT_DIR / "_warnings.log"

# Notebook identity
NOTEBOOK_ID = "nb06"
NOTEBOOK_NAME = "delay_propagation__cascades_and_superspreaders"

# Plotting settings
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("husl")

# Ensure output directories exist
TABLES_REPORT_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_REPORT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Analysis dir exists: {ANALYSIS_DIR.exists()}")

In [ ]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def append_warning(message: str, notebook_id: str = NOTEBOOK_ID):
    """Append a warning to the consolidated warnings log."""
    timestamp = datetime.now().isoformat()
    with open(WARNINGS_LOG, "a") as f:
        f.write(f"[{timestamp}] [{notebook_id}] {message}\n")
    print(f"WARNING: {message}")

def safe_load_parquet(path: Path) -> pl.DataFrame | None:
    """Safely load a parquet file, returning None if it fails."""
    try:
        return pl.read_parquet(path)
    except Exception as e:
        append_warning(f"Failed to load {path.name}: {e}")
        return None

<a id="discover"></a>
## 2. Discover Delay Propagation Artifacts

In [ ]:
# ============================================================================
# DISCOVER DELAY PROPAGATION ARTIFACTS
# ============================================================================

delay_keywords = ["delay", "propagation", "cascade", "spread", "superspreader", "contagion"]

# Search in analysis directory
analysis_files = list(ANALYSIS_DIR.glob("*.parquet")) + list(ANALYSIS_DIR.glob("*.csv")) + list(ANALYSIS_DIR.glob("*.json"))
delay_candidates = [
    f for f in analysis_files 
    if any(kw in f.name.lower() for kw in delay_keywords)
]

print(f"Found {len(delay_candidates)} delay propagation artifacts:")
for df in sorted(delay_candidates):
    print(f"  - {df.name}")

# Primary files
delay_cascades_file = ANALYSIS_DIR / "delay_cascades.parquet"
delay_summary_file = ANALYSIS_DIR / "delay_propagation_summary.json"

print(f"\nDelay cascades exists: {delay_cascades_file.exists()}")
print(f"Delay summary exists: {delay_summary_file.exists()}")

<a id="load"></a>
## 3. Load and Inspect Delay Data

In [ ]:
# ============================================================================
# LOAD AND INSPECT DELAY DATA
# ============================================================================

delay_cascades = None
delay_summary = None

# Load cascades
if delay_cascades_file.exists():
    delay_cascades = safe_load_parquet(delay_cascades_file)
    if delay_cascades is not None:
        print(f"Delay cascades shape: {delay_cascades.shape}")
        print(f"Columns: {delay_cascades.columns}")
        display(delay_cascades.head(10).to_pandas())
else:
    append_warning("delay_cascades.parquet not found")

# Load summary
if delay_summary_file.exists():
    with open(delay_summary_file) as f:
        delay_summary = json.load(f)
    print(f"\nDelay summary:")
    for k, v in delay_summary.items():
        print(f"  {k}: {v}")
else:
    print("\nDelay summary not found")

<a id="cascade-dist"></a>
## 4. Cascade Size Distribution

Analyze the distribution of cascade sizes to check for heavy tails.

In [ ]:
# ============================================================================
# CASCADE SIZE DISTRIBUTION
# ============================================================================

if delay_cascades is not None:
    # Identify cascade size column
    size_col = next((c for c in ["cascade_size", "n_infected", "affected_flights", "size", "total_affected"] 
                     if c in delay_cascades.columns), None)
    
    if size_col:
        print(f"Using cascade size column: {size_col}")
        
        # Compute frequency distribution
        cascade_sizes = delay_cascades[size_col].to_numpy()
        
        # Summary statistics
        print(f"\nCascade Size Statistics:")
        print(f"  Count: {len(cascade_sizes)}")
        print(f"  Min: {cascade_sizes.min()}")
        print(f"  Max: {cascade_sizes.max()}")
        print(f"  Mean: {cascade_sizes.mean():.2f}")
        print(f"  Median: {np.median(cascade_sizes):.0f}")
        print(f"  Std: {cascade_sizes.std():.2f}")
        
        # 90th and 99th percentiles
        p90 = np.percentile(cascade_sizes, 90)
        p99 = np.percentile(cascade_sizes, 99)
        print(f"  90th percentile: {p90:.0f}")
        print(f"  99th percentile: {p99:.0f}")
        
        # Frequency table
        cascade_freq = (
            delay_cascades
            .group_by(size_col)
            .agg(pl.count().alias("frequency"))
            .sort(size_col)
        )
        
        # Plot
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Histogram
        ax1 = axes[0]
        ax1.hist(cascade_sizes, bins=50, edgecolor="white", alpha=0.8, log=True)
        ax1.set_xlabel("Cascade Size")
        ax1.set_ylabel("Frequency (log scale)")
        ax1.set_title("Cascade Size Distribution")
        ax1.axvline(p90, color="orange", linestyle="--", label=f"90th pctl: {p90:.0f}")
        ax1.axvline(p99, color="red", linestyle="--", label=f"99th pctl: {p99:.0f}")
        ax1.legend()
        
        # Log-log plot to check for power law
        ax2 = axes[1]
        freq_df = cascade_freq.to_pandas()
        ax2.scatter(freq_df[size_col], freq_df["frequency"], alpha=0.6, s=20)
        ax2.set_xscale("log")
        ax2.set_yscale("log")
        ax2.set_xlabel("Cascade Size (log)")
        ax2.set_ylabel("Frequency (log)")
        ax2.set_title("Log-Log Plot (Heavy Tail Check)")
        
        plt.tight_layout()
        fig_path = FIGURES_REPORT_DIR / f"{NOTEBOOK_ID}_cascade_size_distribution.png"
        plt.savefig(fig_path, dpi=150)
        plt.show()
        print(f"✅ Saved: {fig_path.name}")
    else:
        append_warning("Could not identify cascade size column")
else:
    print("Not available: delay cascades data not loaded")

<a id="superspreaders"></a>
## 5. Superspreader Ranking

Identify nodes that generate the largest delay cascades.

In [ ]:
# ============================================================================
# SUPERSPREADER RANKING
# ============================================================================

superspreaders = None

if delay_cascades is not None:
    # Identify node/source column
    source_col = next((c for c in ["seed_airport", "source", "origin", "seed_node", "airport"] 
                       if c in delay_cascades.columns), None)
    
    # Identify impact column
    impact_col = next((c for c in ["total_delay_impact", "cascade_size", "total_affected", "impact", "n_infected"] 
                       if c in delay_cascades.columns), None)
    
    if source_col and impact_col:
        print(f"Source column: {source_col}, Impact column: {impact_col}")
        
        # Aggregate impact by source node
        superspreaders = (
            delay_cascades
            .group_by(source_col)
            .agg([
                pl.col(impact_col).sum().alias("total_impact"),
                pl.col(impact_col).mean().alias("mean_impact"),
                pl.col(impact_col).max().alias("max_impact"),
                pl.count().alias("n_cascades")
            ])
            .sort("total_impact", descending=True)
            .with_row_index("rank", offset=1)
            .head(20)
        )
        
        print("\nTOP 20 DELAY SUPERSPREADERS:")
        display(superspreaders.to_pandas())
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 8))
        ss_df = superspreaders.to_pandas()
        
        colors = sns.color_palette("Reds_r", len(ss_df))
        bars = ax.barh(range(len(ss_df)), ss_df["total_impact"], color=colors)
        ax.set_yticks(range(len(ss_df)))
        ax.set_yticklabels(ss_df[source_col])
        ax.invert_yaxis()
        ax.set_xlabel("Total Delay Impact")
        ax.set_title("Top 20 Delay Superspreader Airports")
        
        plt.tight_layout()
        fig_path = FIGURES_REPORT_DIR / f"{NOTEBOOK_ID}_delay_superspreaders_top20.png"
        plt.savefig(fig_path, dpi=150)
        plt.show()
        print(f"✅ Saved: {fig_path.name}")
    else:
        append_warning(f"Could not identify source ({source_col}) or impact ({impact_col}) columns")
else:
    print("Not available: delay cascades data not loaded")

<a id="overlap"></a>
## 6. Centrality Overlap Analysis

Compare superspreaders with structurally central airports.

In [ ]:
# ============================================================================
# CENTRALITY OVERLAP ANALYSIS
# ============================================================================

overlap_df = None

if superspreaders is not None:
    # Load centrality data
    centrality_file = ANALYSIS_DIR / "airport_centrality.parquet"
    
    if centrality_file.exists():
        centrality = safe_load_parquet(centrality_file)
        
        if centrality is not None:
            # Identify matching ID column
            cent_id_col = next((c for c in ["airport", "node", "id"] if c in centrality.columns), None)
            
            if cent_id_col:
                # Get top-20 by betweenness
                betweenness_col = next((c for c in centrality.columns if "betweenness" in c.lower()), None)
                
                if betweenness_col:
                    top_centrality = set(
                        centrality.sort(betweenness_col, descending=True)
                        .head(20)[cent_id_col]
                        .to_list()
                    )
                    top_superspreaders = set(superspreaders[source_col].to_list())
                    
                    overlap = top_centrality & top_superspreaders
                    overlap_rate = len(overlap) / 20
                    
                    print(f"\n📊 CENTRALITY-DELAY OVERLAP ANALYSIS:")
                    print(f"   Top-20 by {betweenness_col}: {len(top_centrality)} airports")
                    print(f"   Top-20 superspreaders: {len(top_superspreaders)} airports")
                    print(f"   Overlap: {len(overlap)} airports ({overlap_rate:.1%})")
                    print(f"   Overlapping airports: {sorted(overlap)}")
                    
                    # Create overlap table
                    overlap_data = []
                    for airport in top_superspreaders:
                        overlap_data.append({
                            "airport": airport,
                            "in_top20_superspreaders": True,
                            "in_top20_centrality": airport in top_centrality
                        })
                    for airport in top_centrality - top_superspreaders:
                        overlap_data.append({
                            "airport": airport,
                            "in_top20_superspreaders": False,
                            "in_top20_centrality": True
                        })
                    overlap_df = pd.DataFrame(overlap_data)
                else:
                    append_warning("Betweenness column not found in centrality data")
            else:
                append_warning("Could not identify ID column in centrality data")
    else:
        print("Centrality data not found - overlap analysis not available")
else:
    print("Not available: superspreaders not computed")

<a id="interpretation"></a>
## 7. Interpretation

### Key Findings (Evidence-Grounded)

*(Populated after running cells above)*

### Mechanistic Explanation

- **Cascade dynamics**: Delays propagate through connecting flights, creating cascades
- **Heavy tails**: Power-law-like distributions suggest rare but extreme disruption events
- **Superspreaders as bridges**: High-betweenness nodes connect many paths, amplifying cascades

### Alternative Explanations
1. Weather events may cause correlated delays at geographic clusters
2. Schedule banking at hubs creates synchronized vulnerability windows
3. Capacity constraints at busy airports amplify initial delays

### Evidence Links
- Figure: `results/figures/report/nb06_cascade_size_distribution.png`
- Table: `results/tables/report/nb06_delay_superspreaders_top20.csv`
- Table: `results/tables/report/nb06_centrality_delay_overlap.csv`

<a id="write-outputs"></a>
## 8. Write Report Outputs

In [ ]:
# ============================================================================
# WRITE REPORT OUTPUTS
# ============================================================================

# Write cascade size distribution
if delay_cascades is not None and size_col:
    cascade_freq_path = TABLES_REPORT_DIR / f"{NOTEBOOK_ID}_cascade_size_distribution.csv"
    cascade_freq.to_pandas().to_csv(cascade_freq_path, index=False)
    print(f"✅ Wrote: {cascade_freq_path}")

# Write superspreaders
if superspreaders is not None:
    ss_path = TABLES_REPORT_DIR / f"{NOTEBOOK_ID}_delay_superspreaders_top20.csv"
    superspreaders.to_pandas().to_csv(ss_path, index=False)
    print(f"✅ Wrote: {ss_path}")

# Write overlap
if overlap_df is not None:
    overlap_path = TABLES_REPORT_DIR / f"{NOTEBOOK_ID}_centrality_delay_overlap.csv"
    overlap_df.to_csv(overlap_path, index=False)
    print(f"✅ Wrote: {overlap_path}")

print(f"\n📋 All {NOTEBOOK_ID} outputs written.")

<a id="reproducibility"></a>
## 9. Reproducibility Notes

### Input Files Consumed
- `results/analysis/delay_cascades.parquet`
- `results/analysis/delay_propagation_summary.json`
- `results/analysis/airport_centrality.parquet` (for overlap analysis)

### Assumptions Made
1. Cascade data comes from IC-model simulation or empirical delay tracking
2. Impact metric represents total propagated delay
3. Overlap analysis uses top-20 threshold for both rankings

### Sorting/Ordering
- Superspreaders ranked by total impact descending
- Tie-breaking by airport code ascending

### Outputs Generated
| Artifact | Path |
|----------|------|
| Cascade Distribution | `results/tables/report/nb06_cascade_size_distribution.csv` |
| Cascade Distribution Figure | `results/figures/report/nb06_cascade_size_distribution.png` |
| Superspreaders Table | `results/tables/report/nb06_delay_superspreaders_top20.csv` |
| Superspreaders Figure | `results/figures/report/nb06_delay_superspreaders_top20.png` |
| Overlap Analysis | `results/tables/report/nb06_centrality_delay_overlap.csv` |